In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

Importá la base de datos.

In [40]:
df = pd.read_excel('Databases/1. Generales - Original.xlsx')

In [41]:
df = df.rename(columns={'autopercep_izq-der': 'autopercep_izq_der'})

Definí las variables independientes y las dependientes. Al ser un modelo ordinal, la variable dependiente debe ser ordinal, o sea, debe tener categorías que pueden ser ordenadas jerárquicamente, como calificaciones (por ejemplo, "mala", "regular", "buena") o niveles de satisfacción (por ejemplo, "insatisfecho", "neutral", "satisfecho") 

In [42]:
# Variables a explicar.
Variables_Dependientes = ['cercania_Massa', 
                          #'cercania_Milei', 
                          #'cercania_Bregman', 
                          #'cercania_Bullrich', 
                          #'cercania_Schiaretti'
                          ]

# Variables que influyen en las otras.
Variables_Todas = [
                            'genero',
                             'edad',
                            'nacionalidad',
                            'provincia',
                            'e_social',
                            'niv_educativo',
                            'voto_2019',
                            'voto_PASO_2023',
                            'candidato_PASO_2023',
                            'autopercep_izq_der',
                            'autopercep_conpro',
                            'autopercep_perantiper',
                            'indice_positividad',
                            'massa_ip_izqder',
                            'massa_ip_conpro',
                             'bullrich_ip_izqder',
                             'bullrich_ip_conpro',
                            'schiaretti_ip_izqder',
                             'schiaretti_ip_conpro',
                            'milei_ip_izqder',
                            'milei_ip_conpro',
                            'bregman_ip_izqder',
                            'bregman_ip_conpro',
                            'indice_progresismo',
                            'indice_conservadurismo'
                            ]

In [43]:
Variables_Massa = [
    'voto_2019',
    'autopercep_perantiper',
    'massa_ip_conpro',
    'indice_conservadurismo',
    'massa_ip_izqder',
    'edad',
    'autopercep_conpro',
    'indice_progresismo',
    'autopercep_izq_der',
    'indice_positividad',
    'candidato_PASO_2023',
    'schiaretti_ip_conpro',
    'schiaretti_ip_izqder'
]

In [44]:
Variables_Schiaretti = [
    'schiaretti_ip_conpro',
    'bullrich_ip_izqder',
    'indice_conservadurismo',
    'provincia',
    'massa_ip_izqder',
    'schiaretti_ip_izqder',
    'candidato_PASO_2023',
    'voto_2019',
    'indice_progresismo'
]

In [45]:
Variables_Independientes = Variables_Massa

In [46]:
X = df[Variables_Independientes]
y = df[Variables_Dependientes]

In [47]:
# Convert categorical independent variables to numerical using LabelEncoder.
for Column in Variables_Independientes:
    if X[Column].dtype == 'object':
        X[Column] = LabelEncoder().fit_transform(X[Column])

C:\Users\tomas\AppData\Local\Temp\ipykernel_10676\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])
C:\Users\tomas\AppData\Local\Temp\ipykernel_10676\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])


In [48]:
# Split the data into training and testing sets
X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
# Ajustar el modelo de regresión logística ordinal sin añadir la constante manualmente.
Ordinal_Model = OrderedModel(y_Train, X_Train, distr='logit')
Ordinal_Result = Ordinal_Model.fit()

C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [50]:
# Verificar la linealidad en el logit (probar la inclusión de términos cuadráticos).
X_Train['edad_sq'] = X_Train['edad'] ** 2  # Ejemplo con la variable 'edad'.

In [51]:
# Ajustar el modelo con término cuadrático.
Ordinal_Model_Non_Linear = OrderedModel(y_Train, X_Train, distr='logit')
Ordinal_Result_Non_Linear = Ordinal_Model_Non_Linear.fit()

C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [52]:
# Resumen del modelo ajustado para verificar los coeficientes de los términos.
print(Ordinal_Result_Non_Linear.summary())

                             OrderedModel Results                             
Dep. Variable:         cercania_Massa   Log-Likelihood:                -2932.0
Model:                   OrderedModel   AIC:                             5900.
Method:            Maximum Likelihood   BIC:                             6003.
Date:                Mon, 14 Oct 2024                                         
Time:                        17:24:01                                         
No. Observations:                2271                                         
Df Residuals:                    2253                                         
Df Model:                          14                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
voto_2019                 -0.0851      0.018     -4.673      0.000      -0.121      -0.049
autopercep_peran

In [53]:
# Realizar predicciones en el set de prueba.
Predictions = Ordinal_Result.predict(X_Test)

In [54]:
# Verificar la proporcionalidad de las odds comparando con un modelo multinomial.
from statsmodels.formula.api import mnlogit

In [55]:
# Ajustar un modelo de regresión logística multinomial para comparación.
Multinomial_Model = mnlogit('cercania_Massa ~ ' + ' + '.join(Variables_Independientes), df)
Multinomial_Result = Multinomial_Model.fit()

# Mostrar el resumen del modelo multinomial para la comparación.
print(Multinomial_Result.summary())

Optimization terminated successfully.
         Current function value: nan
         Iterations 17


C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\discrete\discrete_model.py:3027: RuntimeWarning: overflow encountered in exp
  eXB = np.column_stack((np.ones(len(X)), np.exp(X)))
C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\discrete\discrete_model.py:3028: RuntimeWarning: invalid value encountered in divide
  return eXB/eXB.sum(1)[:,None]


                          MNLogit Regression Results                          
Dep. Variable:         cercania_Massa   No. Observations:                 2839
Model:                        MNLogit   Df Residuals:                     2679
Method:                           MLE   Df Model:                          156
Date:                Mon, 14 Oct 2024   Pseudo R-squ.:                     nan
Time:                        17:24:01   Log-Likelihood:                    nan
converged:                       True   LL-Null:                       -4320.1
Covariance Type:            nonrobust   LLR p-value:                       nan
                                cercania_Massa=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                               nan        nan        nan        nan         nan         nan
voto_2019[T.J. Go

In [56]:
# Revisa si hay valores NaN en el dataset
print(df.isnull().sum())

id                     0
url_exp_id             0
url_environment        0
url_test_version       0
exp_id                 0
                      ..
horassueñopromedio     0
sueñonocheanterior     0
windowresolutionend    0
total                  0
result_165             0
Length: 270, dtype: int64


In [57]:
# Revisa el conteo de valores por categoría
print(df['voto_2019'].value_counts())
print(df['candidato_PASO_2023'].value_counts())

voto_2019
Alberto Fernandez      1568
Mauricio Macri          455
Nicolás Del Caño        286
no vote                 219
Voto en blanco           95
Roberto Lavagna          84
prefiero no decirlo      73
J.L. Espert              48
J. Gomez Centurión       11
Name: count, dtype: int64
candidato_PASO_2023
Sergio Massa                 665
Juan Grabois                 579
Myriam Bregman               386
no aplica                    332
Patricia Bullrich            215
Horacio Rodriguez Larreta    210
Javier Milei                 187
Juan Schiaretti               84
Voto en blanco                67
Guillermo Moreno              33
Prefiero no decirlo           24
Gabriel Solano                22
Manuela Castañeira            21
Marcelo Ramal                  4
Jesús Escobar                  3
Julio Bárbaro                  2
Mempo Giardinelli              1
Santiago Cuneo                 1
votoPASO2023si                 1
votoPASO2023no                 1
Nazareno Etchepare             1